# [Integration of lakeFS with Airflow](https://docs.lakefs.io/integrations/airflow.html)

## Use Case: Troubleshooting production issues

## Import Python packages

In [ ]:
import lakefs_client
import os

## Working with the lakeFS Python client API

###### Note: To learn more about lakeFS Python integration visit https://docs.lakefs.io/integrations/python.html

In [ ]:
%xmode Minimal
from lakefs_client import models
from lakefs_client.client import LakeFSClient

# lakeFS credentials and endpoint
configuration = lakefs_client.Configuration()
configuration.username = lakefsAccessKey
configuration.password = lakefsSecretKey
configuration.host = lakefsEndPoint

client = LakeFSClient(configuration)

## Verify lakeFS credentials by getting lakeFS version

In [ ]:
print("Verifying lakeFS credentials")
client.config.get_lake_fs_version()
print("lakeFS credentials verified")

## Start Airflow

In [4]:
%%script bash --bg --out script_out --err script_error
pkill airflow
airflow standalone

## Create Airflow connections for lakeFS and Spark

In [ ]:
# Wait for Airflow to start
! sleep 10

! airflow connections delete conn_lakefs
lakeFSConnectionCommand = 'airflow connections add conn_lakefs --conn-type=http --conn-host=' + lakefsEndPoint + ' --conn-extra=\'{"access_key_id":"' + lakefsAccessKey + '","secret_access_key":"' + lakefsSecretKey + '"}\''
! $lakeFSConnectionCommand

! airflow connections delete conn_spark
sparkConnectionCommand = 'airflow connections add conn_spark --conn-type=spark --conn-host=local[*]'
! $sparkConnectionCommand

## Set Airflow variables which are used by the demo workflow

In [ ]:
! airflow variables set lakefsAccessKey $lakefsAccessKey
! airflow variables set lakefsSecretKey $lakefsSecretKey
! airflow variables set lakefsEndPoint $lakefsEndPoint
! airflow variables set repo $repo
! airflow variables set sourceBranch $sourceBranch
! airflow variables set newBranch $newBranch
! airflow variables set newPath $newPath
! airflow variables set conn_lakefs 'conn_lakefs'

spark_home = os.getenv('SPARK_HOME')
! airflow variables set spark_home $spark_home